In [2]:
!pip install torch

  Using cached torch-2.5.1-cp312-cp312-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-p

In [4]:
import pandas as pd
import torch
import ast
import os
import utils

DIRPATH_DATA = "Data"
DIRPATH_EXTRACTED_FEATURES = "extracted_features"

brands_in_csv_filename = ["ad",
"brabantsdagblad",
"destentor",
"nu",
"parool",
"trouw",
"volkskrant",]

In [15]:
df = pd.DataFrame()

df.to_csv('../../dataset/test')
df = pd.read_csv('../../dataset/test')

In [17]:
data_pd = pd.concat([pd.read_csv(os.path.join(DIRPATH_DATA, f"uva-relevance-windows-{brand_in_csv_filename}.csv"), sep=";")
                     for brand_in_csv_filename in brands_in_csv_filename], ignore_index=True)


FileNotFoundError: [Errno 2] No such file or directory: 'Data/uva-relevance-windows-ad.csv'

In [4]:
def EmbedS1inE2(values, max):
       values = (values / max) * torch.pi * 2
       out = torch.stack([torch.cos(values), torch.sin(values)], dim=1)
       return out


def to_vector(x, idx):
       d = ast.literal_eval(x)
       vector = torch.zeros(len(idx))
       for topic_str in d:
              vector[idx[topic_str]] = d[topic_str]
       return vector


def to_ohe(column):
       i2s = column.unique().tolist()
       s2i = {s: i for i, s in enumerate(i2s)} 

       one_hot_encoder = torch.eye(len(i2s))
       column_index = [s2i[row] for row in column]
       
       return one_hot_encoder[column_index]


def to_brand_dir(feature_tensor, indices_dict, filename):
       for brand in indices_dict:
              dir_path_brand = os.path.join(DIRPATH_EXTRACTED_FEATURES, brand)
              if not os.path.exists(dir_path_brand):
                     os.makedirs(dir_path_brand)
              file_path = os.path.join(dir_path_brand, filename)
              torch.save(feature_tensor[indices_dict[brand]], file_path)

In [5]:
brands = data_pd["brand"].unique().tolist()

brand_indices = {brand: torch.tensor(data_pd.index[data_pd['brand'] == brand]).int() for brand in brands}

In [8]:
brand_ohe = to_ohe(data_pd["brand"])
to_brand_dir(brand_ohe, brand_indices, utils.FILENAME_BRAND_OHE)


minutes = []
for row in data_pd["publication_timestamp"]:
       minutes.append(int(row[11:13]) * 60 + int(row[14:16]))
minutes = torch.tensor(minutes)

to_brand_dir(minutes.unsqueeze(1), brand_indices, utils.FILENAME_PUBLICATION_TIMESTAMP)

minutes_S1 = EmbedS1inE2(minutes, 60*24)
to_brand_dir(minutes_S1, brand_indices, utils.FILENAME_PUBLICATION_TIMESTAMP_EMB)


weekday_ohe = to_ohe(data_pd["publication_weekday"])
to_brand_dir(weekday_ohe, brand_indices, utils.FILENAME_PUBLICATION_WEEKDAY_OHE)

num_word = torch.tensor(data_pd['num_words']).unsqueeze(1)
to_brand_dir(num_word, brand_indices, utils.FILENAME_NUM_WORDS)

num_par = torch.tensor(data_pd['num_words']).unsqueeze(1)
to_brand_dir(num_par, brand_indices, utils.FILENAME_NUM_PARAGRAPH)


main_ohe = to_ohe(data_pd['main_section'])
to_brand_dir(main_ohe, brand_indices, utils.FILENAME_MAIN_SECTION_OHE)

subsections_ohe = to_ohe(data_pd['subsections'])
to_brand_dir(subsections_ohe, brand_indices, utils.FILENAME_SUBSECTIONS_OHE)

lda_topics = torch.tensor(data_pd["lda_topics"].map(lambda lda_topics: list(ast.literal_eval(lda_topics).values())))
to_brand_dir(lda_topics, brand_indices, utils.FILENAME_LDA_TOPICS)

iptc_level_0 = set()
for row in data_pd["iptc_topics_level_0"]:
       iptc_level_0 = iptc_level_0.union(set(ast.literal_eval(row).keys()))
iptc_level_1 = set()
for row in data_pd["iptc_topics_level_1"]:
       iptc_level_1 = iptc_level_1.union(set(ast.literal_eval(row).keys()))
iptc_level_0_indexed = {topic_str: topic_id for topic_id, topic_str in enumerate(iptc_level_0)}
iptc_level_1_indexed = {topic_str: topic_id for topic_id, topic_str in enumerate(iptc_level_1)}

iptc_0 = torch.stack(data_pd['iptc_topics_level_0'].map(lambda x: to_vector(x, iptc_level_0_indexed)).tolist())
to_brand_dir(iptc_0, brand_indices, utils.FILENAME_IPTC_TOPICS_LEVEL_0)

iptc_1 = torch.stack(data_pd['iptc_topics_level_1'].map(lambda x: to_vector(x, iptc_level_1_indexed)).tolist())
to_brand_dir(iptc_1, brand_indices, utils.FILENAME_IPTC_TOPICS_LEVEL_1)

target = torch.FloatTensor(data_pd['relevance_window_in_hours']).unsqueeze(1)
to_brand_dir(target, brand_indices, utils.FILENAME_RELEVANCE_WINDOW)